In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select




from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
import time
import traceback

In [15]:
import pandas as pd
from datetime import datetime


pos = 0
autentica = pd.read_excel('ls.xlsx')
login = str(autentica['login'][pos])

autentica['senha'] = autentica['senha'].astype(str)
autentica['senha'] = autentica['senha'].str.zfill(11)

senha = autentica['senha'][pos]

display(autentica)
name = str(autentica['ubs'][pos])
data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

raceList = ['BRANCA', 'PRETA', 'AMARELA', 'PARDA', 'INDÍGENA', 'SEM INFORMAÇÃO']


# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav = webdriver.Edge(EdgeChromiumDriverManager().install())


,login,senha,ubs
0,joana,05216791418,borges


In [16]:
def openEpHealph():
    
#     nav = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     nav = webdriver.Chrome(ChromeDriverManager().install())
    global nav
    nav = webdriver.Firefox(executable_path=GeckoDriverManager().install())
    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/login")
    
    # inserindo usuário
    nav.find_element(By.ID, 'EMail').click()
    nav.find_element(By.ID, 'EMail').send_keys(login)

    # inserindo senha 
    nav.find_element(By.ID, 'Password').click()
    nav.find_element(By.ID, 'Password').send_keys(senha)

    nav.find_element(By.XPATH, '/html/body/form/button').click()

In [17]:
def people():
    
    openEpHealph()
        
    print(f'Extração iniciada as: {datetime.now()}')

    nav.get("https://afogadosdaingazeira.ephealth.com.br/portal/individuo")

    children = nav.find_element(By.ID, 'maxAge')
    nav.execute_script("arguments[0].setAttribute('value', '4.0')",children)
    time.sleep(1)
    
#     hypertension = nav.find_element(By.XPATH, '//*[@id="img_hipertensaoFlag"]').click()
#     time.sleep(1)

#     diabetes = nav.find_element(By.XPATH, '//*[@id="img_diabetesFlag"]').click()
#     time.sleep(1)

#     pregnant = nav.find_element(By.XPATH, '//*[@id="img_gestanteFlag"]').click()
#     time.sleep(1)

    """
    race = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[3]/div[5]/span/span[1]/span/ul/li/input')
    race.click()
    time.sleep(1)
    race.send_keys(raceList[race_item])
    race.send_keys(Keys.ENTER)
    """

    time.sleep(1)

    unity = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/form/div[4]/div[1]/span/span[1]/span/span[2]')
    unity.click()
    time.sleep(1)
    nav.find_element(By.XPATH,'/html/body/span/span/span[1]/input').send_keys(Keys.ENTER)
    time.sleep(1)
    nav.find_element(By.ID, 'button').click()

    time.sleep(1)
    nav.find_element(By.ID, 'card-toggle').click()
    time.sleep(10)

In [18]:
def get_table():
    
    list_links = []
    people()
    
    print(f'Iniciando lista de links: {datetime.now()}')

    exit = False
    
    peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
    link = peoples.find_elements(By.TAG_NAME, 'a') 
    
    
    
    while exit is False:
        try:
            for a in link:
                
                list_links.append(a.get_attribute('href'))
                
            page = nav.find_element(By.CSS_SELECTOR, f"#individuo-list-datatable_next > a:nth-child(1)")

            page.click()
            time.sleep(10)                
            
        except (ElementClickInterceptedException, NoSuchElementException):
            if ElementClickInterceptedException:    
                print(ElementClickInterceptedException)
                
            if NoSuchElementException: 
                print(NoSuchElementException)
            print('acabaram as paginas')
            break
        
        except StaleElementReferenceException:
            peoples = nav.find_element(By.CLASS_NAME, 'dataTables_scrollBody')
            link = peoples.find_elements(By.TAG_NAME, 'a') 

    
    list_links = list(dict.fromkeys(list_links))
    print(f'lista de links finalizada as: {datetime.now()}')
    
    return list_links    


In [19]:
def get_individual_info():
    
    individual = get_table()
    
#   name sex b_date cpf cns has dm med_rec race m_name, proffisional, unity
    name, sex, b_date, cpf, cns, med_rec, race, m_name, profissional, unity = [], [], [], [], [], [], [], [], [], []
    
    
    print(f'Iniciando individuos : {datetime.now()}')
    for i in individual:
        
        time.sleep(1)
        
        nav.get(i)
        
        i_name = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/h5').text
        name.append(i_name)
        
        i_sex = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[1]').text
        sex.append(i_sex)
        
        i_bdate = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[4]/td[2]').text
        b_date.append(i_bdate)
        
        i_cpf = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[2]/div/div/table/tbody/tr[2]/td[2]').text
        cpf.append(i_cpf)
        
        i_cns = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        cns.append(i_cns)
        
        i_medrec = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[1]/td[2]').text
        med_rec.append(i_medrec)
        
        i_race = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[5]/td[2]').text
        race.append(i_race.upper())
        
        i_mname = nav.find_element(By.XPATH, '//*[@id="tab-cadastro"]/div/div[3]/div/div/table/tbody/tr[6]/td[2]').text
        m_name.append(i_mname)
        
        i_prof = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[3]/div/div/div/div/div[1]/div/div[2]/div/div/table/tbody/tr[4]/td[2]').text
        profissional.append(i_prof)
        
        i_uni = nav.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]/div[2]/div/div/div/div/div/div/div/div[1]/div/div[2]/p[2]').text
        unity.append(i_uni)
        
        time.sleep(1)

    print(f'Finalizando individuos : {datetime.now()}')

    dict_individual = {"name": name, "sex":sex, "b_date":b_date, "cpf":cpf, "cns":cns, "med_rec":med_rec, "race":race, "m_name":m_name, "profissional":profissional, "unity":unity, "link":individual}
    df_individual = pd.DataFrame(data=dict_individual)    
    
    print(f'Extração finalizada as: {datetime.now()}')    
    return df_individual

# indi = get_indivivual_info()

In [20]:
children = get_individual_info()

Extração iniciada as: 2024-01-23 19:45:13.899255
Iniciando lista de links: 2024-01-23 19:45:30.535994
<class 'selenium.common.exceptions.ElementClickInterceptedException'>
<class 'selenium.common.exceptions.NoSuchElementException'>
acabaram as paginas
lista de links finalizada as: 2024-01-23 19:46:14.049407
Iniciando individuos : 2024-01-23 19:46:14.049438
Finalizando individuos : 2024-01-23 20:01:45.805291
Extração finalizada as: 2024-01-23 20:01:45.808498


In [21]:
display(children)
children = children[['cpf', 'name', 'b_date', 'sex', 'race', 'm_name', 'med_rec', 'link']]

,name,sex,b_date,cpf,cns,med_rec,race,m_name,profissional,unity,link
0,ALICE BEATRIZ ANGELO DA SILVA,"Feminino, 1 ano 4 meses e 1 dia",22/09/2022; Criança,18293990473,CNS: 706405678756281,1880,PARDA,STEFANE RAÍSSA FERREIRA DA SILVA,MARIA ZULEIDE BESERRA CANDIDO,CNS: 706405678756281,https://afogadosdaingazeira.ephealth.com.br/po...
1,ALLANA BEATRIZ PEREIRA DA SILVA,"Feminino, 4 anos 3 meses e 4 dias",19/10/2019; Criança,16772047451,CNS: 706503354178891,--,PARDA,KAROLAINE DE BRITO PEREIRA,MARIA JOSE DE SOUSA ALVES MARQUES,CNS: 706503354178891,https://afogadosdaingazeira.ephealth.com.br/po...
2,ANA CECILIA LIMA NOGUEIRA,"Feminino, 0 ano 5 meses e 30 dias",24/07/2023; Criança,18605669446,CNS: 705007852174359,--,BRANCA,PATRICIA RENISON LIMA NOGUEIRA,ALYSSON TIAGO MARQUES BRANDAO,CNS: 705007852174359,https://afogadosdaingazeira.ephealth.com.br/po...
3,ANA CECILIA RIBEIRO PEREIRA,"Feminino, 4 anos 9 meses e 27 dias",27/03/2019; Criança,16367155406,CNS: 708609572627788,--,BRANCA,SAMARA DA SILVA RIBEIRO,ALYSSON TIAGO MARQUES BRANDAO,CNS: 708609572627788,https://afogadosdaingazeira.ephealth.com.br/po...
4,ANA LIZ BEZERRA DE MORAIS ALVES,"Feminino, 2 anos 3 meses e 25 dias",29/09/2021; Criança,17857876402,CNS: 700506595577658,1985,PARDA,PRISCILA BEZERRA DE MORAIS ALVES,MARIA ZULEIDE BESERRA CANDIDO,CNS: 700506595577658,https://afogadosdaingazeira.ephealth.com.br/po...
...,...,...,...,...,...,...,...,...,...,...,...
243,VICTOR MANOEL DA SILVA FERREIRA,"Masculino, 4 anos 3 meses e 3 dias",20/10/2019; Criança,16792465429,CNS: 700501932992650,1924,BRANCA,JACILENE AVELINA DA SILVA,IVANEIDE OLIVEIRA DOS SANTOS,CNS: 700501932992650,https://afogadosdaingazeira.ephealth.com.br/po...
244,WEVERTON SAULO DA SILVA CANDIDO,"Masculino, 1 ano 0 mês e 3 dias",20/01/2023; Criança,18420754463,CNS: 708604051701683,--,PARDA,SARA FERNANDA DA SILVA CÂNDIDO,PAULA SUZILANEA NICACIO VERAS,CNS: 708604051701683,https://afogadosdaingazeira.ephealth.com.br/po...
245,WITHALY GABRIELLY QUEIROZ SILVA,"Feminino, 0 ano 10 meses e 2 dias",21/03/2023; Criança,18524047445,CNS: 704706764828033,--,PARDA,WITHALY NAIANE PEREIRA DA SILVA BATISTA,MARIA JOSE DE SOUSA ALVES MARQUES,CNS: 704706764828033,https://afogadosdaingazeira.ephealth.com.br/po...
246,YASMIN MENDES BARBOSA,"Feminino, 3 anos 8 meses e 3 dias",20/05/2020; Criança,17196079400,CNS: 704108114110275,--,PARDA,SIMARA MENDES DA SILVA VASCO,PAULA SUZILANEA NICACIO VERAS,CNS: 704108114110275,https://afogadosdaingazeira.ephealth.com.br/po...


In [22]:
display(children)

,cpf,name,b_date,sex,race,m_name,med_rec,link
0,18293990473,ALICE BEATRIZ ANGELO DA SILVA,22/09/2022; Criança,"Feminino, 1 ano 4 meses e 1 dia",PARDA,STEFANE RAÍSSA FERREIRA DA SILVA,1880,https://afogadosdaingazeira.ephealth.com.br/po...
1,16772047451,ALLANA BEATRIZ PEREIRA DA SILVA,19/10/2019; Criança,"Feminino, 4 anos 3 meses e 4 dias",PARDA,KAROLAINE DE BRITO PEREIRA,--,https://afogadosdaingazeira.ephealth.com.br/po...
2,18605669446,ANA CECILIA LIMA NOGUEIRA,24/07/2023; Criança,"Feminino, 0 ano 5 meses e 30 dias",BRANCA,PATRICIA RENISON LIMA NOGUEIRA,--,https://afogadosdaingazeira.ephealth.com.br/po...
3,16367155406,ANA CECILIA RIBEIRO PEREIRA,27/03/2019; Criança,"Feminino, 4 anos 9 meses e 27 dias",BRANCA,SAMARA DA SILVA RIBEIRO,--,https://afogadosdaingazeira.ephealth.com.br/po...
4,17857876402,ANA LIZ BEZERRA DE MORAIS ALVES,29/09/2021; Criança,"Feminino, 2 anos 3 meses e 25 dias",PARDA,PRISCILA BEZERRA DE MORAIS ALVES,1985,https://afogadosdaingazeira.ephealth.com.br/po...
...,...,...,...,...,...,...,...,...
243,16792465429,VICTOR MANOEL DA SILVA FERREIRA,20/10/2019; Criança,"Masculino, 4 anos 3 meses e 3 dias",BRANCA,JACILENE AVELINA DA SILVA,1924,https://afogadosdaingazeira.ephealth.com.br/po...
244,18420754463,WEVERTON SAULO DA SILVA CANDIDO,20/01/2023; Criança,"Masculino, 1 ano 0 mês e 3 dias",PARDA,SARA FERNANDA DA SILVA CÂNDIDO,--,https://afogadosdaingazeira.ephealth.com.br/po...
245,18524047445,WITHALY GABRIELLY QUEIROZ SILVA,21/03/2023; Criança,"Feminino, 0 ano 10 meses e 2 dias",PARDA,WITHALY NAIANE PEREIRA DA SILVA BATISTA,--,https://afogadosdaingazeira.ephealth.com.br/po...
246,17196079400,YASMIN MENDES BARBOSA,20/05/2020; Criança,"Feminino, 3 anos 8 meses e 3 dias",PARDA,SIMARA MENDES DA SILVA VASCO,--,https://afogadosdaingazeira.ephealth.com.br/po...
